# 🚀 Gold Trackr Deploy (GitHub Actions Build)

This notebook downloads the pre-built artifact from GitHub Actions and deploys it.

In [ ]:
# @title 🔧 Configuration
GITHUB_REPO = "ford442/gold_tracker"  # @param {type: "string"}

# Optional: Specific run ID (leave empty for latest)
RUN_ID = ""  # @param {type: "string"}

In [ ]:
# @title ⬇️ Download Latest Build from GitHub Actions
import requests
import os
import zipfile
import json

headers = {
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28"
}

# Add GitHub token if repo is private
# headers["Authorization"] = "Bearer YOUR_GITHUB_TOKEN"

def get_latest_run():
    """Get the latest successful workflow run"""
    if RUN_ID:
        return RUN_ID
    
    url = f"https://api.github.com/repos/{GITHUB_REPO}/actions/runs?branch=main&status=success&per_page=1"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    runs = resp.json()["workflow_runs"]
    if not runs:
        raise Exception("No successful runs found!")
    return runs[0]["id"]

def download_artifact(run_id):
    """Download the build artifact"""
    # First, list artifacts for this run
    url = f"https://api.github.com/repos/{GITHUB_REPO}/actions/runs/{run_id}/artifacts"
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    artifacts = resp.json()["artifacts"]
    
    artifact = None
    for a in artifacts:
        if a["name"] == "goldtrackr-dist":
            artifact = a
            break
    
    if not artifact:
        raise Exception("Build artifact not found! Make sure the workflow completed.")
    
    # Download the artifact
    download_url = artifact["archive_download_url"]
    print(f"Downloading artifact from run {run_id}...")
    
    # Note: This requires GitHub token for private repos
    resp = requests.get(download_url, headers=headers, allow_redirects=True)
    
    if resp.status_code == 404:
        print("⚠️ Artifact not accessible via API.")
        print(f"👉 Download manually from: https://github.com/{GITHUB_REPO}/actions/runs/{run_id}")
        return None
    
    resp.raise_for_status()
    
    # Save and extract
    zip_path = "/content/goldtrackr-dist.zip"
    with open(zip_path, "wb") as f:
        f.write(resp.content)
    
    # Extract
    extract_path = "/content/goldtrackr-dist"
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_path)
    
    print(f"✅ Downloaded and extracted to {extract_path}")
    return extract_path

# Execute
run_id = get_latest_run()
print(f"Latest successful run: {run_id}")
dist_path = download_artifact(run_id)

In [ ]:
# @title 📤 Deploy (Manual Download Method)

# If automatic download doesn't work, manually upload the artifact first:
# 1. Go to https://github.com/ford442/gold_tracker/actions
# 2. Click latest successful run
# 3. Download 'goldtrackr-dist' artifact
# 4. Upload it here using the file manager

import zipfile
import os

UPLOADED_ZIP = "/content/goldtrackr-dist.zip"  # Change if uploaded to different path

if os.path.exists(UPLOADED_ZIP):
    extract_path = "/content/goldtrackr-dist"
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(UPLOADED_ZIP, "r") as z:
        z.extractall(extract_path)
    print(f"✅ Extracted to {extract_path}")
    !ls -la {extract_path}
else:
    print(f"⚠️ Zip file not found at {UPLOADED_ZIP}")
    print("Please upload the artifact manually or run the download cell above.")